# Materi 05: Ekstraksi Fitur (Feature Extraction)
Ekstraksi fitur adalah langkah krusial dalam pengolahan citra yang berfokus pada pengurangan dimensi data sambil tetap mempertahankan informasi penting dari citra asli.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO

# Fungsi unduh gambar sampel
def get_sample_image(url):
    res = requests.get(url)
    return cv2.cvtColor(np.array(Image.open(BytesIO(res.content))), cv2.COLOR_RGB2BGR)

# Menggunakan gambar objek untuk deteksi fitur
img_url = 'https://raw.githubusercontent.com/opencv/opencv/master/samples/data/butterfly.jpg'
image = get_sample_image(img_url)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
print('Library dan Gambar Berhasil Dimuat!')

## 1. Corner Detection (Harris & Shi-Tomasi)
Digunakan untuk mengidentifikasi titik di mana intensitas berubah secara tajam.
- **Harris Corner Detector**: Invarian terhadap rotasi dan pencahayaan.
- **Shi-Tomasi**: Perbaikan dari Harris dengan memeriksa nilai eigen dari matriks.

In [ ]:
# Shi-Tomasi Corner Detection
corners = cv2.goodFeaturesToTrack(gray, 25, 0.01, 10)
corners = np.int0(corners)
img_corners = image.copy()
for i in corners:
    x, y = i.ravel()
    cv2.circle(img_corners, (x, y), 3, (0, 255, 0), -1)

plt.imshow(cv2.cvtColor(img_corners, cv2.COLOR_BGR2RGB)), plt.title('Shi-Tomasi Corners'), plt.axis('off'), plt.show()

## 2. SIFT, SURF, dan ORB
- **SIFT**: Invarian terhadap skala dan rotasi.
- **SURF**: Alternatif SIFT yang lebih cepat menggunakan matriks Hessian.
- **ORB**: Kombinasi cepat dari FAST dan BRIEF yang efisien.

In [ ]:
# Inisialisasi ORB detector (SIFT/SURF mungkin butuh instalasi khusus di beberapa versi OpenCV)
orb = cv2.ORB_create()
keypoints, descriptors = orb.detectAndCompute(gray, None)

# Gambar keypoints
img_orb = cv2.drawKeypoints(image, keypoints, None, color=(0, 255, 0), flags=0)

plt.imshow(cv2.cvtColor(img_orb, cv2.COLOR_BGR2RGB)), plt.title('ORB Keypoints'), plt.axis('off'), plt.show()

## 3. Histogram of Oriented Gradients (HOG)
Sangat berguna untuk deteksi objek (seperti manusia) dengan menghitung orientasi gradien pada porsi lokal citra.

In [ ]:
from skimage.feature import hog
from skimage import exposure

fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)

hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

plt.imshow(hog_image_rescaled, cmap='gray'), plt.title('HOG Visualization'), plt.axis('off'), plt.show()